# Using Cython to Make Stationary Distribution Function Faster

In [3]:
%load_ext cython

tmax = 10.0
zmax = 10.0
d=1.0

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [4]:
import numpy as np


# define default coefficients
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 
r = 0.02   # Specific mineralization rate of sedimented nutrients

def I(z,A,I_0=I0,k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    integral = np.zeros(len(z))
    integral[1:] = np.cumsum(k*A[1:]) 
    return I_0 * np.exp( - integral- kbg*z)

def p(I,q):
    return mumax * (1.0 - qmin/q) * (I/(h + I))

def rho(q, Rd):
    return rhomax * (qmax-q)/(qmax-qmin) * ( Rd/(m + Rd) )



def next_step(z,A, Rb, Rd, Rs,dz,dt,d):
    """Calculates next step for input arrays of length zmax"""
    
    II = I(z,A)
    q = Rb[1:-1]/A[1:-1]
    pp = p(II[1:-1],q)
    rrho = rho(q,Rd[1:-1])
    
    A_next = np.zeros(len(A))
    Rb_next = np.zeros(len(Rb))
    Rd_next = np.zeros(len(Rd))
    
    A_drift = v * (A[2:]-A[:-2]) / (2*dz)
    A_diffusion = d * (A[2:]-2*A[1:-1] + A[:-2]) / (dz**2)
    Rb_drift = v * (Rb[2:]-Rb[:-2]) / (2*dz)
    Rb_diffusion = d * (Rb[2:]-2*Rb[1:-1] + Rb[:-2]) / (dz**2)
    Rd_diffusion = d * (Rd[2:]-2*Rd[1:-1] + Rd[:-2]) / (dz**2)
    
    A_next[1:-1] = A[1:-1] + dt * ( pp*A[1:-1] -lbg*A[1:-1] - A_drift + A_diffusion )
    A_next[0] = 4*d/(2*v*dz + 3*d)*A_next[1] - d/(2*v*dz + 3*d)*A_next[2] 
    A_next[-1] = (4*A_next[-2] - A_next[-3])/3
    
    Rb_next[1:-1] = Rb[1:-1] + dt * (rrho*A[1:-1] -lbg*Rb[1:-1] - Rb_drift + Rb_diffusion )
    Rb_next[0] = 4*d/(2*v*dz + 3*d)*Rb_next[1] - d/(2*v*dz + 3*d)*Rb_next[2] 
    Rb_next[-1] = (4*Rb_next[-2] - Rb_next[-3])/3
    
    Rs_next = Rs + dt*(v*Rb[-1] - r*Rs)
    
    Rd_next[1:-1] = Rd[1:-1] + dt*(lbg*Rb[1:-1] -rrho*A[1:-1] + Rd_diffusion)
    Rd_next[0] = (4*Rd_next[1] - Rd_next[2])/3 
    Rd_next[-1] = (2*r*dz*Rs_next + 4*d*Rd_next[-2] - d*Rd_next[-3])/(3*d)
    
    
    return A_next, Rb_next, Rd_next, Rs_next
    
def get_stationary(zmax=10, tmax=100, d=1.0, I0=300.0):
    """ A, Rb, Rd, Rs, z_grid, Nz = equations.get_stationary(zmax=10.0, d=1.0, I0=300.0) """
    
    dz = 0.1
    dt = dz/1000 # temporary 
    Nz = int(zmax/dz)
    Nt = int(tmax/ (1000*dt) )

    z_grid = np.arange(0,zmax,dz)
    time_steps = np.arange(0,tmax,dt)

    A_0 = 100 
    Rb_0 = 2.2
    Rd_0 = 30 
    Rs_0 = 0
    
    A = np.zeros((Nz,Nt)) # RESULTS MATRIX: rows: deeper z-values, cols: time steps forward
    A[:,0] = A_0 * np.ones(Nz) # creates homogenous initial conditions

    Rb = np.zeros((Nz,Nt))
    Rb[:,0] = Rb_0 * np.ones(Nz)

    Rd = np.zeros((Nz,Nt))
    Rd[:,0] = Rd_0 * np.ones(Nz)
    
    Rs = np.zeros(Nt)

    A_next,Rb_next,Rd_next, Rs_next = A[:,0], Rb[:,0], Rd[:,0], Rs[0] #initial conditions

    i = 1
    counter = 1
    for t in time_steps[:-1]:
        
        A_next, Rb_next, Rd_next, Rs_next = next_step(z_grid, A_next, Rb_next, Rd_next, Rs_next, dz,dt,d)
        
        if ( i % 1000 == 0):
            A[:,counter],Rb[:,counter],Rd[:,counter],Rs[counter] = A_next,Rb_next,Rd_next,Rs_next
            counter = counter +1
            
        i = i+1
        
    return A[:,-1], Rb[:,-1], Rd[:,-1], Rs[-1]


In [5]:
%%time
A, Rb, Rd, Rs = get_stationary(zmax=zmax, tmax=tmax, d=d)

# CPU times: user 9.3 s, sys: 118 ms, total: 9.42 s
# Wall time: 9.56 s

CPU times: user 9.3 s, sys: 118 ms, total: 9.42 s
Wall time: 9.56 s


In [6]:
%%cython -a

import numpy as np


# define default coefficients
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 
r = 0.02   # Specific mineralization rate of sedimented nutrients

def I(z,A,I_0=I0,k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    integral = np.zeros(len(z))
    integral[1:] = np.cumsum(k*A[1:]) 
    return I_0 * np.exp( - integral- kbg*z)

def p(I,q):
    return mumax * (1.0 - qmin/q) * (I/(h + I))

def rho(q, Rd):
    return rhomax * (qmax-q)/(qmax-qmin) * ( Rd/(m + Rd) )



def next_step(z,A, Rb, Rd, Rs,dz,dt,d):
    """Calculates next step for input arrays of length zmax"""
    
    II = I(z,A)
    q = Rb[1:-1]/A[1:-1]
    pp = p(II[1:-1],q)
    rrho = rho(q,Rd[1:-1])
    
    A_next = np.zeros(len(A))
    Rb_next = np.zeros(len(Rb))
    Rd_next = np.zeros(len(Rd))
    
    A_drift = v * (A[2:]-A[:-2]) / (2*dz)
    A_diffusion = d * (A[2:]-2*A[1:-1] + A[:-2]) / (dz**2)
    Rb_drift = v * (Rb[2:]-Rb[:-2]) / (2*dz)
    Rb_diffusion = d * (Rb[2:]-2*Rb[1:-1] + Rb[:-2]) / (dz**2)
    Rd_diffusion = d * (Rd[2:]-2*Rd[1:-1] + Rd[:-2]) / (dz**2)
    
    A_next[1:-1] = A[1:-1] + dt * ( pp*A[1:-1] -lbg*A[1:-1] - A_drift + A_diffusion )
    A_next[0] = 4*d/(2*v*dz + 3*d)*A_next[1] - d/(2*v*dz + 3*d)*A_next[2] 
    A_next[-1] = (4*A_next[-2] - A_next[-3])/3
    
    Rb_next[1:-1] = Rb[1:-1] + dt * (rrho*A[1:-1] -lbg*Rb[1:-1] - Rb_drift + Rb_diffusion )
    Rb_next[0] = 4*d/(2*v*dz + 3*d)*Rb_next[1] - d/(2*v*dz + 3*d)*Rb_next[2] 
    Rb_next[-1] = (4*Rb_next[-2] - Rb_next[-3])/3
    
    Rs_next = Rs + dt*(v*Rb[-1] - r*Rs)
    
    Rd_next[1:-1] = Rd[1:-1] + dt*(lbg*Rb[1:-1] -rrho*A[1:-1] + Rd_diffusion)
    Rd_next[0] = (4*Rd_next[1] - Rd_next[2])/3 
    Rd_next[-1] = (2*r*dz*Rs_next + 4*d*Rd_next[-2] - d*Rd_next[-3])/(3*d)
    
    
    return A_next, Rb_next, Rd_next, Rs_next
    
def get_stationary(zmax=10, tmax=100, d=1.0, I0=300.0):
    """ A, Rb, Rd, Rs, z_grid, Nz = equations.get_stationary(zmax=10.0, d=1.0, I0=300.0) """
    
    dz = 0.1
    dt = dz/1000 # temporary 
    Nz = int(zmax/dz)
    Nt = int(tmax/ (1000*dt) )

    z_grid = np.arange(0,zmax,dz)
    time_steps = np.arange(0,tmax,dt)

    A_0 = 100 
    Rb_0 = 2.2
    Rd_0 = 30 
    Rs_0 = 0
    
    A = np.zeros((Nz,Nt)) # RESULTS MATRIX: rows: deeper z-values, cols: time steps forward
    A[:,0] = A_0 * np.ones(Nz) # creates homogenous initial conditions

    Rb = np.zeros((Nz,Nt))
    Rb[:,0] = Rb_0 * np.ones(Nz)

    Rd = np.zeros((Nz,Nt))
    Rd[:,0] = Rd_0 * np.ones(Nz)
    
    Rs = np.zeros(Nt)

    A_next,Rb_next,Rd_next, Rs_next = A[:,0], Rb[:,0], Rd[:,0], Rs[0] #initial conditions

    i = 1
    counter = 1
    for t in time_steps[:-1]:
        
        A_next, Rb_next, Rd_next, Rs_next = next_step(z_grid, A_next, Rb_next, Rd_next, Rs_next, dz,dt,d)
        
        if ( i % 1000 == 0):
            A[:,counter],Rb[:,counter],Rd[:,counter],Rs[counter] = A_next,Rb_next,Rd_next,Rs_next
            counter = counter +1
            
        i = i+1
        
    return A[:,-1], Rb[:,-1], Rd[:,-1], Rs[-1]



In [7]:
%%time
A, Rb, Rd, Rs = get_stationary(zmax=zmax, tmax=tmax, d=d)

# CPU times: user 9.19 s, sys: 162 ms, total: 9.35 s
# Wall time: 9.56 s

# Not really any faster

CPU times: user 9.19 s, sys: 162 ms, total: 9.35 s
Wall time: 9.56 s


In [17]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp

from cpython cimport array
import array

cdef array.array float_array_template = array.array('f', [])

cdef array.array scalar_multiply(array.array x, float y, int arrayLength):
    cdef array.array result 
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y*x[i]
    
    return result

cdef array.array vector_multiply(array.array x, array.array y, int arrayLength):
    cdef array.array result 
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y[i]*x[i]
    
    return result

cdef array.array scalar_divide(array.array x, float y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y/x[i]
    return result

cdef array.array vector_divide(array.array x, array.array y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = x[i]/y[i]
    return result

cdef array.array scalar_add(array.array x, float y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y+x[i]
    return result

cdef array.array scalar_subtract(array.array x, float y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y-x[i]
    return result

cdef array.array vector_add(array.array x, array.array y, arrayLength):
    cdef array.array result
    result = array.clone(x, arrayLength, zero=True)
    for i in range(0,arrayLength):
        result[i] = y[i]+x[i]
    return result  
    
cdef float v, lbg, mumax, rhomax, qmax, qmin, m, h, I0, kbg, A_0, Rb_0, Rd_0
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(int Nz, array.array z, array.array A, float I_0=I0, float k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    
    cdef array.array integral = array.clone(A, Nz, zero=True)
    cdef int i = 0 
    
    integral[0] = k*A[0]
    
    for i in range(Nz):
        i = i + 1
        integral[i] = k*integral[i-1]
    
    return scalar_multiply(exp( - integral -  scalar_multiply(z, kbg, Nz)),I0,Nz)

def p(int Nz,array.array I,array.array q):
    
    cdef array.array result1 = array.clone(I, Nz, zero=True)
    cdef array.array result2 = array.clone(I, Nz, zero=True)
    cdef array.array result = array.clone(I, Nz, zero=True)
    
    result1 = scalar_subtract(scalar_divide(q,qmin,Nz),1.0,Nz)
    result2 = vector_divide(I,scalar_add(I,h,Nz),Nz)
    result = vector_multiply(result1,result2,Nz)
    result = scalar_multiply(result,mumax,Nz)
    
    return result

def rho(Nz, array.array q, array.array Rd):
    
    cdef array.array result1 = array.clone(Rd, Nz, zero=True)
    cdef array.array result2 = array.clone(Rd, Nz, zero=True)
    cdef array.array result = array.clone(Rd, Nz, zero=True)
    
    result1 = vector_divide(scalar_subtract(q,qmin,Nz),scalar_subtract(q,qmax,Nz),Nz)
    result2 = vector_divide(Rd,vector_add(Rd,m,Nz),Nz)
    result = vector_multiply(result1,result2,Nz)
    result = scalar_multiply(result,rhomax,Nz)
    
    return result


## for testing

def create_arrays(int Nz):
    cdef array.array zeros, ones, twos
    zeros = array.clone(float_array_template, Nz, zero=True)
    ones = scalar_add(zeros,1,Nz)
    twos = scalar_add(zeros,2,Nz)
    return zeros, ones, twos

In [18]:
ones,twos,threes = create_arrays(10)
len(ones)

10

In [13]:
ones + twos # arrays not so good

array('f', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [19]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp

ctypedef np.int32_t cINT32
ctypedef np.float_t cFLOAT

cdef np.ndarray[cFLOAT, ndim=1] float_array_template 


Error compiling Cython file:
------------------------------------------------------------
...
from libc.math cimport exp

ctypedef np.int32_t cINT32
ctypedef np.float_t cFLOAT

cdef np.ndarray[cFLOAT, ndim=1] float_array_template 
                               ^
------------------------------------------------------------

/Users/alisonpeard/.ipython/cython/_cython_magic_be2822eb6a932d7afb16f866733e50bc.pyx:9:32: Buffer types only allowed as function local variables

Error compiling Cython file:
------------------------------------------------------------
...

import numpy as np
^
------------------------------------------------------------

/Users/alisonpeard/.ipython/cython/_cython_magic_be2822eb6a932d7afb16f866733e50bc.pyx:2:0: Buffer vars not allowed in module scope


In [15]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp

ctypedef np.int32_t cINT32
ctypedef np.float_t cFLOAT

cdef float v, lbg, mumax, rhomax, qmax, qmin, m, h, I0, kbg, A_0, Rb_0, Rd_0
v= 0.25    # drift term
lbg=0.1    # Specific algal maintenance respiration losses
mumax=1.2  # Maximum specific algal production rate
rhomax=0.2 # Maximum specific algal nutrient uptake rate
qmax=0.04  # Maximum algal nutrient quota
qmin=0.004 # Minimum algal nutrient quota 
m=1.5      # Half-saturation constant of algal nutrient uptake
h=120.0    # Half-saturation constant of light-dependent algal production
I0 = 300   # Light intensity at the surface 
kbg=0.4    # Background light-attenuation coefficient 

def I(np.ndarray[cFLOAT, ndim=1] z, np.ndarray[cFLOAT, ndim=1] A, float I_0=I0, float k = 0.0003):
    """Function to plot I using array A[:,i], default k=0.0003, larger values of k make effect of A on I more apparent"""
    
    cdef np.ndarray[cFLOAT, ndim=1] integral
    cdef int i = 0
    
    integral = np.zeros(len(z))   ## NUMPY
    
    integral[0] = k*A[1]
    
    for i in range(len(z)):
        i = i + 1
        integral[i] = k*integral[i-1]
    
    return I_0 * exp( - integral - kbg*z)

array([0, 2, 4, 6, 8])